# Parallel Data Enrichment with DuckDB

This notebook demonstrates how to use the `parallel-web-tools` package to enrich DuckDB tables using the Parallel API.

## Features

- **Batch Processing**: Efficient parallel enrichment of entire tables
- **SQL UDF**: Row-by-row enrichment via SQL functions
- **Multiple processors**: Choose speed vs. depth tradeoff
- **Error handling**: Graceful handling with detailed error reporting

## Prerequisites

```bash
pip install parallel-web-tools[duckdb]
export PARALLEL_API_KEY="your-api-key"
```

## Setup

In [15]:
# Install dependencies if needed
# !pip install parallel-web-tools[duckdb]

In [16]:
import json

import duckdb

from parallel_web_tools.integrations.duckdb import enrich_table, register_parallel_functions

print(f"DuckDB version: {duckdb.__version__}")

DuckDB version: 1.4.3


## Authentication

Set your Parallel API key via environment variable or pass it directly.

In [17]:
import os

from dotenv import load_dotenv

# Load environment variables from .env file (if present)
load_dotenv()

api_key = os.environ.get("PARALLEL_API_KEY")
if api_key:
    print(f"PARALLEL_API_KEY is set ({len(api_key)} chars)")
else:
    print("PARALLEL_API_KEY not found. Create a .env file with:")
    print("  PARALLEL_API_KEY=your-key")

PARALLEL_API_KEY is set (40 chars)


## Create Sample Data

In [18]:
# Create a DuckDB connection and sample data
conn = duckdb.connect()

conn.execute("""
    CREATE TABLE companies AS SELECT * FROM (VALUES
        ('Google', 'google.com', 'Technology'),
        ('Microsoft', 'microsoft.com', 'Technology'),
        ('Apple', 'apple.com', 'Technology'),
        ('Amazon', 'amazon.com', 'E-commerce'),
        ('Parallel Web Systems', 'paralell.ai', 'Technology')
    ) AS t(company_name, website, industry)
""")

conn.execute("SELECT * FROM companies").fetchdf()

,company_name,website,industry
0,Google,google.com,Technology
1,Microsoft,microsoft.com,Technology
2,Apple,apple.com,Technology
3,Amazon,amazon.com,E-commerce
4,Parallel Web Systems,paralell.ai,Technology


## Basic Batch Enrichment (Recommended)

Batch processing is the most efficient approach for multiple rows.

In [19]:
# Enrich with CEO name and founding year
# Note: This will make API calls - may take a few seconds

result = enrich_table(
    conn,
    source_table="SELECT company_name, website FROM companies LIMIT 2",
    input_columns={
        "company_name": "company_name",
        "website": "website",
    },
    output_columns=[
        "CEO name (current CEO or equivalent leader)",
        "Founding year (YYYY format)",
        "Brief company description (1-2 sentences)",
    ],
)

print(f"Success: {result.success_count}, Errors: {result.error_count}")
print(f"Time: {result.elapsed_time:.2f} seconds")
result.result.fetchdf()

Success: 2, Errors: 0
Time: 13.49 seconds


,company_name,website,ceo_name,founding_year,brief_company_description
0,Google,google.com,Sundar Pichai,1998,Google is an American search engine company fo...
1,Microsoft,microsoft.com,Satya Nadella,1975,Microsoft is an American multinational technol...


## Understanding the Result

The `EnrichmentResult` object contains:
- `relation`: DuckDB relation with enriched data
- `success_count`: Number of rows successfully enriched
- `error_count`: Number of rows that failed
- `errors`: List of error details for failed rows
- `elapsed_time`: Total processing time

In [20]:
# Check for any errors
if result.error_count > 0:
    print("Errors encountered:")
    for error in result.errors:
        print(f"  Row {error['row']}: {error['error']}")
else:
    print("All rows enriched successfully!")

All rows enriched successfully!


## Column Name Mapping

Output columns are automatically converted to valid SQL identifiers:

| Description | Column Name |
|-------------|-------------|
| `"CEO name"` | `ceo_name` |
| `"Founding year (YYYY)"` | `founding_year` |
| `"Brief company description"` | `brief_company_description` |

In [21]:
# See the column names
print("Enriched columns:", result.result.columns)

Enriched columns: ['company_name', 'website', 'ceo_name', 'founding_year', 'brief_company_description']


## Working with Results

The result is a DuckDB relation that can be queried further or converted to a DataFrame.

In [22]:
# Convert to pandas DataFrame
df = result.result.fetchdf()
df[["company_name", "ceo_name", "founding_year"]]

,company_name,ceo_name,founding_year
0,Google,Sundar Pichai,1998
1,Microsoft,Satya Nadella,1975


In [23]:
# Or use SQL on the relation
result.result.filter("founding_year IS NOT NULL").select("company_name, founding_year").fetchdf()

,company_name,founding_year
0,Google,1998
1,Microsoft,1975


## Progress Tracking

For large batches, you can track progress with a callback.

In [24]:
def progress_callback(completed: int, total: int):
    pct = 100 * completed / total if total > 0 else 0
    print(f"\rProgress: {completed}/{total} ({pct:.0f}%)", end="")


# result = enrich_table(
#     conn,
#     source_table="companies",
#     input_columns={"company_name": "company_name"},
#     output_columns=["CEO name"],
#     progress_callback=progress_callback,
# )
# print()  # New line after progress

## Including Citations (Basis)

You can include the sources used for enrichment by setting `include_basis=True`.

In [25]:
# Get enrichment with citations
result_with_basis = enrich_table(
    conn,
    source_table="SELECT company_name FROM companies LIMIT 1",
    input_columns={"company_name": "company_name"},
    output_columns=["CEO name"],
    include_basis=True,
)

# Access the basis (citations)
df = result_with_basis.result.fetchdf()
for _, row in df.iterrows():
    print(f"Company: {row['company_name']}")
    print(f"CEO: {row['ceo_name']}")
    print(f"Sources: {row['_basis']}")

Company: Google
CEO: Sundar Pichai
Sources: [{'field': ceo_name, 'citations': [{'url': 'https://en.wikipedia.org/wiki/Sundar_Pichai', 'excerpts': [CEO of Alphabet Inc. and Google since 2015]}], 'reasoning': 'The user asks for the current CEO of Google. The search results include a Wikipedia entry for Sundar Pichai that explicitly states his role as CEO of both Alphabet Inc. and its subsidiary Google. That matches the requested information, so the answer is Sundar Pichai.', 'confidence': high}]


## Creating Permanent Tables

You can save enriched results to a permanent table.

In [26]:
# Save results to a permanent table
# result = enrich_table(
#     conn,
#     source_table="companies",
#     input_columns={"company_name": "company_name"},
#     output_columns=["CEO name"],
#     result_table="enriched_companies",  # Creates permanent table
# )
#
# # Query it later
# conn.execute("SELECT * FROM enriched_companies").fetchdf()

## Processor Options

Choose a processor based on your needs:

| Processor | Speed | Cost | Best For |
|-----------|-------|------|----------|
| `lite-fast` | Fastest | Lowest | Basic metadata, high volume |
| `base-fast` | Fast | Low | Standard enrichments |
| `core-fast` | Medium | Medium | Cross-referenced data |
| `pro-fast` | Slow | High | Deep research |

In [27]:
# Use a different processor for more depth
result_detailed = enrich_table(
    conn,
    source_table="SELECT company_name FROM companies LIMIT 1",
    input_columns={"company_name": "company_name"},
    output_columns=[
        "Recent news headline about this company",
        "Stock ticker symbol",
    ],
    processor="base-fast",  # Use base processor for more depth
)

result_detailed.result.fetchdf()

,company_name,recent_news_headline_about_this_company,stock_ticker_symbol
0,Google,Google settles Google Assistant privacy lawsui...,GOOGL


## SQL UDF Approach (Alternative)

For simple queries or when you prefer SQL, you can use the registered UDF.
Note: This is slower than batch processing for multiple rows.

In [28]:
# Register the UDF
register_parallel_functions(conn, processor="lite-fast")

# Use in a query (one row only for demo)
results = conn.execute("""
    SELECT
        company_name,
        parallel_enrich(
            json_object('company_name', company_name),
            json_array('CEO name')
        ) as enriched
    FROM companies
    LIMIT 1
""").fetchall()

# Parse the JSON result
for name, enriched_json in results:
    data = json.loads(enriched_json)
    print(f"{name}: {data}")

/var/folders/m5/w3whc94944763szbd_pqg_0h0000gn/T/ipykernel_34483/1643809355.py:5: RuntimeWarning: coroutine '_enrich_all_async' was never awaited
  results = conn.execute("""


InvalidInputException: Invalid Input Error: Python exception occurred while executing the UDF: RuntimeError: asyncio.run() cannot be called from a running event loop

At:
  /Users/mattharris/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/asyncio/runners.py(191): run
  /Users/mattharris/workspace/parallel-web/parallel-tools/parallel_web_tools/integrations/duckdb/udf.py(137): _enrich_batch_sync
  /Users/mattharris/workspace/parallel-web/parallel-tools/parallel_web_tools/integrations/duckdb/udf.py(219): enrich_vectorized
  /var/folders/m5/w3whc94944763szbd_pqg_0h0000gn/T/ipykernel_34483/1643809355.py(5): <module>
  /Users/mattharris/workspace/parallel-web/parallel-tools/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py(3701): run_code
  /Users/mattharris/workspace/parallel-web/parallel-tools/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py(3641): run_ast_nodes
  /Users/mattharris/workspace/parallel-web/parallel-tools/.venv/lib

## Error Handling

Errors in individual rows don't stop the batch processing. Failed rows will have NULL values in enriched columns.

In [ ]:
# Create a table with potential errors
conn.execute("""
    CREATE TABLE companies_with_issues AS SELECT * FROM (VALUES
        ('Google'),
        ('NonexistentCompanyXYZ123')
    ) AS t(company_name)
""")

result = enrich_table(
    conn,
    source_table="companies_with_issues",
    input_columns={"company_name": "company_name"},
    output_columns=["CEO name"],
)

print(f"Success: {result.success_count}, Errors: {result.error_count}")

# Check errors
if result.errors:
    for error in result.errors:
        print(f"Row {error['row']}: {error['error']}")

# View results (errors show as NULL)
result.result.fetchdf()

## Best Practices

### 1. Use Batch Processing for Multiple Rows

```python
# Good - uses parallel Task Group API
result = enrich_table(conn, "companies", ...)

# Slower - one API call per row
conn.execute("SELECT *, parallel_enrich(...) FROM companies")
```

### 2. Be Specific in Descriptions

```python
# Good - specific descriptions
output_columns = [
    "CEO name (current CEO or equivalent leader)",
    "Founding year (YYYY format)",
    "Annual revenue (USD, most recent fiscal year)",
]

# Less specific - may get inconsistent results
output_columns = ["CEO", "Year", "Revenue"]
```

### 3. Use Appropriate Processors

- `lite-fast`: Basic metadata, high volume (cheapest)
- `base-fast`: Standard company information
- `pro-fast`: Deep research requiring multiple sources

### 4. Handle Errors Gracefully

```python
result = enrich_table(conn, ...)
if result.error_count > 0:
    logger.warning(f"{result.error_count} rows failed")
```

## Next Steps

- See the [DuckDB Setup Guide](../docs/duckdb-setup.md) for more details
- Check [Parallel Documentation](https://docs.parallel.ai) for API information
- View [parallel-web-tools on GitHub](https://github.com/parallel-web/parallel-web-tools)